In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from sklearn import svm
import math

## NaiveBayes implementation in Sklearn to analyze diabetes risk.

In [3]:
# Training data
data_train=pd.read_csv("HW2_task4_train.csv") 
y_train=data_train.iloc[:,1] 
X_train=data_train.iloc[:,2:] 

# Testing data
data_test=pd.read_csv("HW2_task4_test.csv")
y_test=data_test.iloc[:,1]
X_test=data_test.iloc[:,2:]

print(X_train.head())

   t_pre    glu  blood_p  triceps  serum   b_m  pedigree_f   age
0    3.0  158.0     64.0     13.0  387.0  31.2       0.295  24.0
1    0.0   84.0     64.0     22.0   66.0  35.8       0.545  21.0
2    9.0  120.0     72.0     22.0   56.0  20.8       0.733  48.0
3    4.0  110.0     76.0     20.0  100.0  28.4       0.118  27.0
4    2.0  100.0     66.0     20.0   90.0  32.9       0.867  28.0


In [5]:
#train classifier
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [8]:
print("In sample accuracy score = {}".format(gnb.score(X_train,y_train)))
print("Out of sample accuracy score = {}".format(gnb.score(X_test,y_test)))

In sample accuracy score = 0.7584745762711864
Out of sample accuracy score = 0.7911392405063291


In [17]:
print("The prior probability of diabetes (i.e. y-train class probability) is = {}%"
      .format(gnb.class_prior_[1] * 100))

The prior probability of diabetes (i.e. y-train class probability) is = 30.08474576271186%


### Mean and variance of patients with diabetes by factor

In [43]:
for j in range(len(X_train.columns)):
    print("variance of {} for patients with diabetes = {}"
          .format(X_train.columns[j],gnb.sigma_[1][j]))

variance of t_pre for patients with diabetes = 12.206320287887335
variance of glu for patients with diabetes = 924.7689070762225
variance of blood_p for patients with diabetes = 187.7389527020909
variance of triceps for patients with diabetes = 93.96787553486216
variance of serum for patients with diabetes = 15882.053969563818
variance of b_m for patients with diabetes = 38.61704435057331
variance of pedigree_f for patients with diabetes = 0.190055013933758
variance of age for patients with diabetes = 103.8567864652331


In [44]:
for j in range(len(X_train.columns)):
    print("mean value of {} for patients with diabetes = {}"
          .format(X_train.columns[j],gnb.theta_[1][j]))

mean value of t_pre for patients with diabetes = 4.070422535211268
mean value of glu for patients with diabetes = 144.14084507042253
mean value of blood_p for patients with diabetes = 74.56338028169014
mean value of triceps for patients with diabetes = 33.478873239436616
mean value of serum for patients with diabetes = 209.2112676056338
mean value of b_m for patients with diabetes = 35.22394366197182
mean value of pedigree_f for patients with diabetes = 0.6390422535211271
mean value of age for patients with diabetes = 35.7887323943662


### Mean and Variance analysis for patients without diabetes, by factor

In [45]:
for j in range(len(X_train.columns)):
    print("variance of {} for patients without diabetes = {}"
          .format(X_train.columns[j],gnb.sigma_[0][j]))

variance of t_pre for patients without diabetes = 6.771141492303129
variance of glu for patients without diabetes = 605.9943554500628
variance of blood_p for patients without diabetes = 136.36967225447037
variance of triceps for patients without diabetes = 108.26910292481006
variance of serum for patients without diabetes = 8318.987854072659
variance of b_m for patients without diabetes = 39.921929371091004
variance of pedigree_f for patients without diabetes = 0.0846144012471254
variance of age for patients without diabetes = 72.44482376962172


In [46]:
for j in range(len(X_train.columns)):
    print("mean value of {} for patients with diabetes = {}"
          .format(X_train.columns[j],gnb.theta_[0][j]))

mean value of t_pre for patients with diabetes = 2.690909090909091
mean value of glu for patients with diabetes = 111.46666666666667
mean value of blood_p for patients with diabetes = 69.2060606060606
mean value of triceps for patients with diabetes = 27.2
mean value of serum for patients with diabetes = 127.0060606060606
mean value of b_m for patients with diabetes = 31.709090909090893
mean value of pedigree_f for patients with diabetes = 0.46868484848484815
mean value of age for patients with diabetes = 28.393939393939394


## Supervised and Semi-Supervised NB implementation

In [15]:
data_train=pd.read_csv("HW2_task5_train.csv")
y_Labeled_train=data_train.iloc[:,1] 
X_Labeled_train=data_train.iloc[:,2:] 

data_test=pd.read_csv("HW2_task5_test.csv")
y_Labeled_test=data_test.iloc[:,1]
X_Labeled_test=data_test.iloc[:,2:]

data_Unlabeled=pd.read_csv("HW2_task5_unlabeled.csv")
X_Unlabeled_train=data_Unlabeled.iloc[:,1:]

print(X_Labeled_train)
print(y_Labeled_train)

   0  1  2
0  4  6  6
1  2  2  2
2  2  6  2
3  6  6  6
4  1  2  2
5  3  4  4
6  6  1  6
7  1  3  1
8  5  1  6
0    0
1    1
2    0
3    0
4    1
5    0
6    0
7    1
8    1
Name: y, dtype: int64


### Supervised Multi-nomial NB

First I'm experimenting with an Sklearn package that is suitable for discrete naive bayes problems and then I use a UDF.

In [50]:
from sklearn.naive_bayes import MultinomialNB

In [51]:
discrete_nb = MultinomialNB()
discrete_nb.fit(X_Labeled_train,y_Labeled_train)
print("Out of sample labeled classification accuracy: {}"
      .format(discrete_nb.score(X_Labeled_test,y_Labeled_test)))

Out of sample labeled classification accuracy: 0.5


#### Below is a UDF version of NB

In [17]:
# Training a binary Naive Bayes Classifier with discrete input attributes.
# Assume that the binary output variable takes on values 0 or 1. 
def trainNaiveBayesDiscrete(X,y):
    prior = 1.*y.sum()/y.count()
    nbc = {'prior':prior}
    X_1 = X[y==1]
    X_0 = X[y==0]
    for j in X.columns:
        nbc[j+'_1'] = X_1[j].value_counts(normalize=True)
        nbc[j+'_0'] = X_0[j].value_counts(normalize=True)
    return nbc

def testNaiveBayesDiscrete(X,nbc):
    y_pred = pd.Series(index=X.index)
    for i in X.index:
        # compute odds of y=1
        y_pred[i] = nbc['prior']/(1-nbc['prior']) # prior odds
        for j in X.columns:
            thevalue = X.loc[i,j]
            if thevalue not in nbc[j+'_1']:
                y_pred[i] = y_pred[i]*1E-3
            if thevalue not in nbc[j+'_0']:
                y_pred[i] = y_pred[i]*1E3
            if (thevalue in nbc[j+'_1']) & (thevalue in nbc[j+'_0']):
                y_pred[i] = y_pred[i]*(nbc[j+'_1'][thevalue]+1E-3)/(nbc[j+'_0'][thevalue]+1E-3)
        # convert odds to probability of y=1
        y_pred[i] = y_pred[i]/(1.0+y_pred[i])
    return y_pred

In [57]:
naive_bayes_classifier = trainNaiveBayesDiscrete(X_Labeled_train,y_Labeled_train)
y_pred_test = testNaiveBayesDiscrete(X_Labeled_test,naive_bayes_classifier)

# measure accuracy for the binary prediction task
print('Out of sample prediction accuracy:',
      1.0*sum((y_pred_test>0.5)==y_Labeled_test)/len(y_Labeled_test))

Out of sample prediction accuracy: 0.6944444444444444


### Semi-Supervised

In [18]:
def initializeNaiveBayesRandom(X_Unlabeled):
    nbc = {'prior':0.5}
    for j in X_Unlabeled.columns:
        thevalues = X_Unlabeled[j].unique()
        nbc[j+'_1'] = {}
        nbc[j+'_0'] = {}
        for jj in thevalues:
            nbc[j+'_1'][jj] = np.random.rand()
            nbc[j+'_0'][jj] = np.random.rand()
    return nbc
    
def EM(X_Labeled,y_Labeled,X_Unlabeled,num_iters):
    # initialize
    t = 0
    if len(y_Labeled) > 0:
        nbc = trainNaiveBayesDiscrete(X_Labeled,y_Labeled)
    else:
        nbc = initializeNaiveBayesRandom(X_Unlabeled)
    
    while True:
        t = t + 1
        print('Iteration',t,'of',num_iters)
        
        # E step - classify with nbc for unlabeled data only
        y_pred_Unlabeled = testNaiveBayesDiscrete(X_Unlabeled,nbc)
        
        # M step
        X_for_M_step = pd.concat([X_Labeled,X_Unlabeled],ignore_index=True) 
        y_for_M_step = pd.concat([y_Labeled,y_pred_Unlabeled],ignore_index=True)
        prior = 1.*y_for_M_step.sum()/y_for_M_step.count()
        nbc = {'prior':prior}
        for j in X_for_M_step.columns:
            nbc[j+'_1'] = {}
            nbc[j+'_0'] = {}
            for theindex in X_for_M_step.index:
                current_X = X_for_M_step.loc[theindex,j]
                current_y = y_for_M_step.loc[theindex]
                if current_X in nbc[j+'_1']:
                    nbc[j+'_1'][current_X] += current_y
                else:
                    nbc[j+'_1'][current_X] = current_y
                if current_X in nbc[j+'_0']:
                    nbc[j+'_0'][current_X] += (1.0-current_y)
                else:
                    nbc[j+'_0'][current_X] = 1.0-current_y
            # normalize probabilities
            tempsum = 0.0
            for k in nbc[j+'_1']:
                tempsum += nbc[j+'_1'][k]
            for k in nbc[j+'_1']:
                nbc[j+'_1'][k] /= tempsum
            tempsum = 0.0
            for k in nbc[j+'_0']:
                tempsum += nbc[j+'_0'][k]
            for k in nbc[j+'_0']:
                nbc[j+'_0'][k] /= tempsum            
                       
        if t==num_iters:
            break
            
    return nbc

In [20]:
semi_supervised = EM(X_Labeled_train,y_Labeled_train,X_Unlabeled_train,10)
y_pred_test = testNaiveBayesDiscrete(X_Labeled_test,semi_supervised)
print('Out of sample prediction accuracy:',
      1.0*sum((y_pred_test>0.5)==y_Labeled_test)/len(y_Labeled_test))

Iteration 1 of 10
Iteration 2 of 10
Iteration 3 of 10
Iteration 4 of 10
Iteration 5 of 10
Iteration 6 of 10
Iteration 7 of 10
Iteration 8 of 10
Iteration 9 of 10
Iteration 10 of 10
Out of sample prediction accuracy: 0.9444444444444444
